In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', 200)

In [2]:
df = pd.read_pickle('FT-data-DSpace/data_processed.pickle')
df.head(1)

,ID,Date,Meeting Number,Agenda Name,Agenda Number,File Type,File Number,File Step,Speaker Name,Speaker Role,Speaker Start,Speaker End,Speech
0,CBAEA326-B201-41AB-A756-029BA1F58D82,2009-10-30T10:00:00,9. møde,[None],0,Formandens meddelelser,,,None,formand,2009-10-30T10:00:18.063,2009-10-30T10:01:04.997,"Mødet er åbnet.I dag er der følgende anmeldelser: Thor Pedersen (V), Mogens Lykketoft (S), Søren Espersen (DF), Holger K. Nielsen (SF) og Helge Adam Møller (KF):Lovforslag nr. L 53 (Forslag til lo..."


In [3]:
df = df[['Date', 'Meeting Number', 'Speech']]
df = df.dropna()
df.sample(2)

,Date,Meeting Number,Speech
129697,2012-08-20T13:00:00,99. møde,"Jeg har i dag hørt Liberal Alliance tale om, at man skulle lade være med at ansætte 8.000 offentligt ansatte. Jeg er af den opfattelse, at det er et dumt forslag. Et stykke arbejde i den offentlig..."
291418,2015-11-04T13:00:00,11. møde,"Tak for det. Jeg kan egentlig svare ret kort: Regeringen prioriterer ældreområdet massivt, men det er velkendt, at vi ikke er varme tilhængere af minimumsrettigheder i ældreplejen, så min holdning..."


In [15]:
df_aggregated = df.groupby(by =['Date', 'Meeting Number'])['Speech'].apply(lambda x: ',,'.join(x)) \
                  .reset_index()
df_aggregated.sample(2)

,Date,Meeting Number,Speech
553,2014-10-30T10:00:00,9. møde,"Mødet er åbnet.I dag er der følgende anmeldelser:Klima-, energi- og bygningsministeren (Rasmus Helveg Petersen):Lovforslag nr. L 11 (Forslag til lov om ændring af lov om CO2-kvoter og lov om kuldi..."
372,2013-01-29T13:00:00,52. møde,Mødet er åbnet.Der er i dag følgende anmeldelse:Jørgen Arbo-Bæhr (EL) m.fl.:Beslutningsforslag nr. B 55 (Forslag til folketingsbeslutning om at efterlønsbidrag og fleksydelsesbidrag udbetalt på gu...


In [16]:
#### TOKENIZATION AND STEMMING
import nltk
from nltk.stem.snowball import DanishStemmer

stopwords = nltk.corpus.stopwords.words('danish') 
tokenizer = nltk.RegexpTokenizer(r"\w+")
stemmer = DanishStemmer()

In [17]:
# tokenize the string and remove all punktuations
df_aggregated['Unstemmed'] = df_aggregated['Speech'].apply(lambda x: tokenizer.tokenize(x)) 

In [18]:
df_aggregated.sample(3)

,Date,Meeting Number,Speech,Unstemmed
787,2016-12-02T10:00:00,27. møde,"Mødet er åbnet, men jeg er måske nødt til at udsætte det.Kommer der en minister nu? Ja.Så starter jeg med at sige, at der i dag er følgende anmeldelser:Henrik Sass Larsen (S), Søren Gade (V), Leif...","[Mødet, er, åbnet, men, jeg, er, måske, nødt, til, at, udsætte, det, Kommer, der, en, minister, nu, Ja, Så, starter, jeg, med, at, sige, at, der, i, dag, er, følgende, anmeldelser, Henrik, Sass, L..."
255,2012-01-20T10:00:00,37. møde,"Mødet er åbnet.Jeg håber, at alle medlemmerne har hørt udråbningen gamle dages vægtere 'udråbte' tiden;-), da vi stadig væk har problemer med ringesystemet.I dag er der følgende anmeldelse:Christi...","[Mødet, er, åbnet, Jeg, håber, at, alle, medlemmerne, har, hørt, udråbningen, gamle, dages, vægtere, udråbte, tiden, da, vi, stadig, væk, har, problemer, med, ringesystemet, I, dag, er, der, følge..."
639,2015-05-21T10:00:00,95. møde,"Mødet er åbnet.Medlemmer af Folketinget Jan E. Jørgensen (V), Dennis Flydtkjær (DF) og Brian Mikkelsen (KF) m.fl. har meddelt mig, at de ønsker at tage følgende forespørgsel tilbage:Hvad kan minis...","[Mødet, er, åbnet, Medlemmer, af, Folketinget, Jan, E, Jørgensen, V, Dennis, Flydtkjær, DF, og, Brian, Mikkelsen, KF, m, fl, har, meddelt, mig, at, de, ønsker, at, tage, følgende, forespørgsel, ti..."


In [19]:
### stem the tokens
df_aggregated['Stemmed'] = df_aggregated['Unstemmed'].apply(lambda x: [stemmer.stem(y) for y in x if y.isalnum()])

In [20]:
## remove stop words and numbers
df_aggregated['Clean'] = df_aggregated['Stemmed'].apply(lambda x: [y for y in x if not y.isnumeric() and not y in stopwords])

In [21]:
### data aggregated
df_aggregated.to_pickle('FT-data-DSpace/data_agreggated.pickle')
df_aggregated.sample(2)

,Date,Meeting Number,Speech,Unstemmed,Stemmed,Clean
432,2013-06-26T12:00:00,112. møde,"Mødet er åbnet.,,Der er ikke mere at foretage i dette møde.Folketingets næste møde afholdes i morgen, torsdag den 27. juni 2013, kl. 10.00.Jeg henviser til den dagsorden, der fremgår af Folketinge...","[Mødet, er, åbnet, Der, er, ikke, mere, at, foretage, i, dette, møde, Folketingets, næste, møde, afholdes, i, morgen, torsdag, den, 27, juni, 2013, kl, 10, 00, Jeg, henviser, til, den, dagsorden, ...","[mød, er, åbn, der, er, ikk, mer, at, foretag, i, det, mød, folketing, næst, mød, afhold, i, morg, torsdag, den, 27, juni, 2013, kl, 10, 00, jeg, henvis, til, den, dagsord, der, fremgår, af, folke...","[mød, åbn, ikk, mer, foretag, mød, folketing, næst, mød, afhold, morg, torsdag, juni, kl, henvis, dagsord, fremgår, folketing, hjemmesid, ønsk, nog, udtal, ikk, tilfæld, så, forhandling, slut, går..."
136,2010-12-02T10:00:00,28. møde,Mødet er åbnet.I dag er der følgende anmeldelser:Justitsministeren (Lars Barfoed):Lovforslag nr. L 89 (Forslag til lov om ændring af lov om forbrugerbeskyttelse ved erhvervelse af fast ejendom m.v...,"[Mødet, er, åbnet, I, dag, er, der, følgende, anmeldelser, Justitsministeren, Lars, Barfoed, Lovforslag, nr, L, 89, Forslag, til, lov, om, ændring, af, lov, om, forbrugerbeskyttelse, ved, erhverve...","[mød, er, åbn, i, dag, er, der, følg, anmeld, justitsminist, lar, barfoed, lovforslag, nr, l, 89, forslag, til, lov, om, ændring, af, lov, om, forbrugerbeskyt, ved, erhverv, af, fast, ejendom, m, ...","[mød, åbn, dag, følg, anmeld, justitsminist, lar, barfoed, lovforslag, nr, l, forslag, lov, ændring, lov, forbrugerbeskyt, ved, erhverv, fast, ejendom, m, v, lov, omsætning, fast, ejendom, lov, au..."
